In [2]:
import arcpy
import os
import imageio.v2 as imageio
import shutil

# Environment settings
arcpy.env.overwriteOutput = True

# List of TIFF images
image_files = [
    "C:/EsriTraining/Ruang_land_cover_change/ruang_0318.tif",
    "C:/EsriTraining/Ruang_land_cover_change/ruang_0427.tif",
    "C:/EsriTraining/Ruang_land_cover_change/ruang_0512.tif",
    "C:/EsriTraining/Ruang_land_cover_change/ruang_0527.tif",
    "C:/EsriTraining/Ruang_land_cover_change/ruang_0530.tif"
]

# Output directory for timelapse
output_directory = "C:/EsriTraining/Ruang_land_cover_change/ruang_output_directory"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Function to clear the output directory
def clear_output_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

# Clear the output directory before exporting new frames
clear_output_directory(output_directory)

# Load the current project and map
try:
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    map = aprx.listMaps()[0]
except Exception as e:
    print("Failed to load the project or map. Reason:", e)
    quit()

# Check if the map view is available
if aprx.activeView is None:
    print("Map view is not available.")
else:
    # Function to add and remove layers
    def update_layer(image_path):
        # Remove all existing layers
        for lyr in map.listLayers():
            map.removeLayer(lyr)
        
        # Add the new layer
        new_layer = arcpy.management.MakeRasterLayer(image_path, "temp_layer")
        map.addLayer(new_layer.getOutput(0))
    
    # Export each frame
    frame_count = 0
    for image_path in image_files:
        update_layer(image_path)
        
        # Refresh the active view
        map_view = aprx.activeView
        if hasattr(map_view, 'exportToPNG'):
            # Get the extent of the first layer in the map
            layer_extent = arcpy.Describe(map.listLayers()[0]).extent
            
            # Set the map view extent
            map_view.camera.setExtent(layer_extent)
            
            # Calculate width and height for export
            width = layer_extent.width
            height = layer_extent.height
            
            # Export to a frame image with higher resolution
            frame_path = os.path.join(output_directory, f"ruang_{frame_count:03d}.png")
            map_view.exportToPNG(frame_path, width=int(width), height=int(height), resolution=600)  # Adjust the resolution as needed
            frame_count += 1
        else:
            print("Map view is not available.")

    print("Frames exported successfully.")

    # Create a time-lapse GIF using imageio
    # Get list of frame images
    frame_images = [os.path.join(output_directory, f) for f in sorted(os.listdir(output_directory)) if f.startswith('ruang_') and f.endswith('.png')]

    # Create a GIF with a lower frame rate to slow down the timelapse
    gif_path = os.path.join(output_directory, "timelapse.gif")
    with imageio.get_writer(gif_path, mode='I', fps=1) as writer:
        for _ in range(5):  # Repeat the timelapse 5 times
            for frame_image in frame_images:
                image = imageio.imread(frame_image)
                writer.append_data(image)

    print("Time-lapse GIF created successfully.")

Frames exported successfully.
Time-lapse GIF created successfully.
